# Proyecto: Clasificación de Desechos Sólidos utilizando Xception (Final)

# Nombre: Pedro Arriola
# Carnet: 20188
---

## 1. Introducción

En este proyecto, nuestro objetivo es crear un modelo de **clasificación de desechos sólidos** utilizando la arquitectura **Xception**, una red neuronal convolucional profunda preentrenada en el conjunto de datos **ImageNet**. La idea principal es construir un modelo capaz de clasificar imágenes de desechos sólidos en diferentes categorías como `battery`, `glass`, `metal`, `organic`, `paper` y `plastic`.

El problema de la clasificación de desechos sólidos es crucial para el manejo de residuos y el reciclaje eficiente. Un sistema automatizado que pueda clasificar correctamente diferentes tipos de desechos puede ayudar a mejorar la eficiencia en los procesos de reciclaje y reducir el impacto ambiental.

Este proyecto aborda las siguientes etapas:
1. Uso del modelo **Xception** preentrenado.
2. Implementación de técnicas de **regularización** (L2) y **dropout** para mejorar la generalización.
3. Realización de **fine-tuning** en el modelo para ajustar sus capas superiores y mejorar su rendimiento.
4. Evaluación de los resultados del modelo utilizando métricas como **precision**, **recall** y **f1-score**, además de una **matriz de confusión**.

---

## 2. ¿Qué es el modelo Xception?

**Xception** es un modelo convolucional profundo basado en una arquitectura de redes neuronales que se destaca por su uso eficiente de convoluciones separables en profundidad (depthwise separable convolutions). Fue desarrollado como una mejora del modelo Inception y se ha demostrado que ofrece un rendimiento superior en tareas de clasificación de imágenes.

La arquitectura de Xception se caracteriza por:
- **Convoluciones separables en profundidad**: Estas convoluciones descomponen una convolución estándar en dos operaciones: una convolución depthwise seguida de una convolución pointwise. Esto reduce el número de parámetros y mejora la eficiencia computacional.
- **Capas profundas**: Xception tiene muchas capas de convolución, lo que le permite aprender características complejas de las imágenes.
- **Preentrenamiento en ImageNet**: Xception está preentrenado en el conjunto de datos **ImageNet**, lo que significa que ya ha aprendido a reconocer características visuales generales como bordes, texturas y formas, lo que facilita su adaptación a nuevas tareas mediante el proceso de **transfer learning**.

En este proyecto, aprovechamos este modelo preentrenado y ajustamos sus capas superiores para adaptarlo a la tarea específica de clasificación de desechos sólidos.
---
## 3. Preprocesamiento de las Imágenes

El preprocesamiento de las imágenes es una etapa crucial en cualquier tarea de clasificación de imágenes, especialmente cuando se utilizan modelos preentrenados como **Xception**. Cada modelo preentrenado tiene su propia función de preprocesamiento específica que ajusta las imágenes para que coincidan con los datos con los que se entrenó el modelo original.

### 3.1 Preprocesamiento en el Modelo Xception

Para el modelo **Xception**, se utiliza la función de preprocesamiento `preprocess_input`. Esta función realiza varias transformaciones importantes en las imágenes antes de alimentarlas al modelo:

- **Escalado**: Las imágenes se escalan para que sus valores estén en el rango de [-1, 1] (en lugar del rango estándar de [0, 255] para los valores de píxeles). Este escalado específico coincide con los datos utilizados durante el preentrenamiento de Xception en ImageNet.
- **Normalización**: La función `preprocess_input` normaliza los valores de los píxeles, ayudando al modelo a tener entradas más estables, lo que a su vez mejora la convergencia durante el entrenamiento.

### 3.2 Ajuste del Tamaño de las Imágenes

Otra transformación importante fue ajustar las imágenes al tamaño requerido por el modelo **Xception**, que es **299x299 píxeles**. Las redes convolucionales preentrenadas como Xception requieren que las imágenes de entrada tengan un tamaño específico para que coincidan con la estructura de la red.

### 3.3 Generación de Imágenes

Para el preprocesamiento y la generación de imágenes durante el entrenamiento, se utilizó la clase `ImageDataGenerator` de Keras. Esta clase permite aplicar transformaciones como el escalado y facilita el procesamiento eficiente de las imágenes en lotes, lo que permite entrenar el modelo en grandes conjuntos de datos. Además, es posible aplicar **data augmentation** utilizando esta clase para generar más variabilidad en los datos, mejorando así la capacidad del modelo para generalizar.

### 3.4 Explicación del Preprocesamiento

1. **Preprocesamiento Específico de Xception**:
   - La función `preprocess_input` escala los valores de los píxeles a un rango de [-1, 1], que es el formato esperado por el modelo Xception.
   - Esto asegura que el modelo reciba las imágenes en un formato compatible con lo que se utilizó durante su preentrenamiento en ImageNet, lo que mejora el rendimiento del modelo.

2. **Data Augmentation (Opcional)**:
   - Además del preprocesamiento básico, también se puede aplicar **data augmentation** para mejorar la capacidad del modelo de generalizar a imágenes nuevas.
   - Algunas técnicas comunes de data augmentation incluyen rotaciones, cambios de escala, ajustes de brillo y traslaciones. Estas técnicas pueden aplicarse a las imágenes durante el entrenamiento para aumentar la variabilidad del conjunto de datos.

### 3.5 Importancia del Preprocesamiento

El preprocesamiento adecuado es esencial para garantizar que las imágenes se presenten al modelo en el formato correcto. Dado que **Xception** fue entrenado en ImageNet con imágenes preprocesadas de una manera específica, replicar este preprocesamiento es clave para aprovechar al máximo las capacidades del modelo preentrenado.

En resumen, para el modelo **Xception**, el preprocesamiento incluyó:
- Escalado de las imágenes al rango [-1, 1] utilizando la función de preprocesamiento adecuada.
- Ajuste de las imágenes al tamaño de **299x299 píxeles** para cumplir con el formato esperado por el modelo.
- Generación de las imágenes de entrenamiento y validación utilizando técnicas de preprocesamiento eficientes, con la posibilidad de aplicar **data augmentation** para aumentar la variabilidad y robustez del conjunto de datos.

---

## 4. Construcción del Modelo

El modelo que construimos utiliza **Xception** como base preentrenada, con capas adicionales densas para clasificar las imágenes en nuestras 6 clases de desechos sólidos. Como parte del proceso de construcción del modelo, se congelaron las capas inferiores de Xception para conservar las características previamente aprendidas y se añadieron capas densas y de regularización.

### Detalles de la Construcción del Modelo:

1. **Base del modelo (Xception)**: Cargamos el modelo preentrenado Xception sin las capas superiores (`include_top=False`). Esto nos permite utilizar las características visuales aprendidas previamente en el conjunto de datos ImageNet.
   
2. **Capas densas adicionales**:
   - Se añadieron dos capas densas: la primera con **1024 unidades** y la segunda con **512 unidades**, ambas activadas con la función `ReLU`.
   - Para evitar el sobreajuste (*overfitting*), se aplicó **regularización L2** a ambas capas densas, con un coeficiente de regularización de **0.002**.
   - Se utilizó **dropout** con una tasa del **60%** para reducir la posibilidad de que el modelo dependa demasiado de ciertas características durante el entrenamiento.

3. **Capa de salida**: Se añadió una capa de salida con **6 unidades** (una para cada clase) activada con la función `softmax`, lo que permite clasificar las imágenes en una de las seis categorías de desechos sólidos.

### Hiperparámetros utilizados:

- **Batch size**: 64
- **Epochs**: 200 (con early stopping)
- **Regularización L2**: 0.002
- **Dropout**: 0.6
- **Optimización**: Adam con `learning_rate=0.0001`
- **Callbacks**: EarlyStopping y ReduceLROnPlateau

Se implementaron los siguientes *callbacks*:
- **EarlyStopping**: para detener el entrenamiento si no se veía mejora en la pérdida de validación durante 15 épocas.
- **ReduceLROnPlateau**: para reducir la tasa de aprendizaje si no había mejora en la pérdida de validación tras 5 épocas.

---

## 5. Fine-tuning del Modelo

Después de entrenar el modelo con las capas de Xception congeladas, realizamos un proceso de **fine-tuning** para ajustar las capas superiores del modelo base. Esto permite que el modelo ajuste mejor las características aprendidas específicamente a la tarea de clasificación de desechos sólidos.

El fine-tuning se realizó descongelando las últimas **50 capas** del modelo Xception, permitiendo que estas capas se ajusten durante el proceso de entrenamiento con un *learning rate* más bajo (`1e-5`).

----
## 6. Resultados Obtenidos

Después de aplicar **regularización L2** y **fine-tuning**, obtuvimos los siguientes resultados en el conjunto de validación:

| Clase     | Precision | Recall  | F1-Score | Soporte |
|-----------|-----------|---------|----------|---------|
| battery   | 1.00      | 0.99    | 0.99     | 87      |
| glass     | 0.87      | 0.93    | 0.90     | 73      |
| metal     | 0.94      | 0.96    | 0.95     | 69      |
| organic   | 0.97      | 1.00    | 0.98     | 64      |
| paper     | 0.98      | 0.99    | 0.98     | 92      |
| plastic   | 0.93      | 0.84    | 0.88     | 80      |

**Precisión Global**: 0.95

**Macro Promedio**:
- **Precision**: 0.95
- **Recall**: 0.95
- **F1-Score**: 0.95

**Promedio Ponderado**:
- **Precision**: 0.95
- **Recall**: 0.95
- **F1-Score**: 0.95

### Observaciones:
- **Precisión global del 95%**: El modelo ha logrado una precisión global alta, mostrando que es capaz de generalizar bien a datos no vistos.
- **Clases con mejor rendimiento**: Las clases "battery", "organic" y "paper" tienen un rendimiento excelente, con *f1-scores* cercanos al 0.99.
- **Clases más difíciles**: La clase "plastic" mostró un *recall* ligeramente más bajo (84%), lo que sugiere que el modelo tiene alguna dificultad para distinguir correctamente algunas imágenes de plástico.

----

## 7. Conclusiones

- El uso de la arquitectura **Xception** junto con **regularización L2** y **dropout** ha permitido obtener un **alto rendimiento** en la clasificación de desechos sólidos, con una precisión global del **95%**.
  
- El **fine-tuning** de las capas superiores del modelo base permitió ajustar las características aprendidas a la tarea específica, mejorando aún más el rendimiento del modelo.

- Aunque el rendimiento general es excelente, la clase "plastic" aún muestra cierto grado de confusión con un *recall* del 84%. Para mejorar esta clase, podría ser útil aplicar **data augmentation** adicional (rotaciones, zoom, cambios de brillo, etc.) o ajustar más finamente el modelo con un mayor enfoque en esta clase.

- El modelo es suficientemente robusto para su uso en aplicaciones de clasificación de desechos sólidos, lo que puede contribuir a mejorar los procesos de reciclaje y gestión de residuos de manera más eficiente.



In [ ]:
!pip install -q seaborn

In [ ]:
from tensorflow.keras.applications.xception import Xception, preprocess_input
from tensorflow.keras import layers
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.layers import Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras import regularizers
from tensorflow.keras.preprocessing import image
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix, classification_report
from PIL import Image
from collections import Counter
import os

import warnings
warnings.filterwarnings('ignore')

## EDA

In [ ]:
data_path = '/content/drive/MyDrive/Colab_Notebooks/EcoScan/trash'

In [ ]:
# Revisa si el path existe y recopila información de las clases
if os.path.exists(data_path):
    # Obtiene los nombres de las clases y cuenta las imágenes en cada clase
    class_names = [d for d in os.listdir(data_path) if os.path.isdir(os.path.join(data_path, d))]
    class_counts = {class_name: len(os.listdir(os.path.join(data_path, class_name))) for class_name in class_names}
    total_images = sum(class_counts.values())

    # Información resumen
    print("Resumen del conjunto de datos:")
    print(f"Total de clases: {len(class_names)}")
    print(f"Total de imágenes: {total_images}")
    print("Distribución de clases:")
    for class_name, count in class_counts.items():
        print(f"  - {class_name}: {count} imágenes")
else:
    print("La ruta de datos no existe.")

## MODEL CONSTRUCTION

In [ ]:
# Parámetros
BATCH_SIZE = 32
n_classes = 6

In [ ]:
# Cargar el modelo base preentrenado Xception
conv_base = Xception(
    include_top=False,
    weights='imagenet',
    input_shape=(299, 299, 3)  # Tamaño ajustado a 299x299 para Xception
)

In [ ]:
# Congelar todas las capas de Xception excepto las dos últimas
for layer in conv_base.layers:
    layer.trainable = False  # Congelar todas las capas inicialmente

In [ ]:
# Congelar todas las capas excepto las dos últimas
for layer in conv_base.layers[:-2]:
    layer.trainable = True

In [ ]:
# Valor de la regularización L2 (puedes ajustarlo)
l2_value = 0.002

# Construir el modelo superior (top model) con regularización L2
top_model = conv_base.output
top_model = Flatten(name="flatten")(top_model)
top_model = Dense(1024, activation='relu', kernel_regularizer=regularizers.l2(l2_value))(top_model)  # Regularización L2
top_model = BatchNormalization()(top_model)
top_model = Dropout(0.6)(top_model)
top_model = Dense(512, activation='relu', kernel_regularizer=regularizers.l2(l2_value))(top_model)  # Regularización L2
top_model = BatchNormalization()(top_model)
top_model = Dropout(0.6)(top_model)
output_layer = Dense(n_classes, activation='softmax')(top_model)  # Capa de salida

# Modelo final
model = Model(inputs=conv_base.input, outputs=output_layer)

In [ ]:
# Función para mostrar imágenes aleatorias del dataset
def plot_random_images(generator, num_images=20):
    images, labels = next(generator)
    random_indices = np.random.choice(images.shape[0], num_images, replace=False)
    fig, axs = plt.subplots(4, 5, figsize=(15, 12))
    fig.suptitle('Random 20 Images from the Generator', fontsize=16)
    for i, ax in enumerate(axs.flatten()):
        index = random_indices[i]
        image = images[index]
        label = labels[index]
        ax.imshow(image)
        ax.set_title(f'Class: {np.argmax(label)}')
        ax.axis('off')
    plt.show()

In [ ]:
# Preparar los generadores de datos
original_data = ImageDataGenerator(rescale=1./255).flow_from_directory(
    data_path, target_size=(299, 299), batch_size=BATCH_SIZE, class_mode="categorical")
plot_random_images(original_data)

gen_train = ImageDataGenerator(preprocessing_function=preprocess_input)  # Preprocesamiento para Xception
full_data = gen_train.flow_from_directory(data_path, target_size=(299, 299), batch_size=BATCH_SIZE, class_mode="categorical")
plot_random_images(full_data)

In [ ]:
# Extraer nombres de archivos y etiquetas
filenames = full_data.filenames
labels = full_data.labels
class_mapping = {value: str(key) for key, value in full_data.class_indices.items()}
labels = [class_mapping[label] for label in labels]

# Dividir en entrenamiento y validación
train_filenames, test_filenames, train_labels, test_labels = train_test_split(
    filenames, labels, test_size=0.1, random_state=42)

# Crear DataFrames para entrenamiento y validación
train_df = pd.DataFrame({'filename': train_filenames, 'class': train_labels})
test_df = pd.DataFrame({'filename': test_filenames, 'class': test_labels})

# Crear generadores de entrenamiento y validación
train_data = gen_train.flow_from_dataframe(
    train_df, directory=data_path, target_size=(299, 299),
    batch_size=BATCH_SIZE, class_mode="categorical", shuffle=True, seed=42)

test_data = gen_train.flow_from_dataframe(
    test_df, directory=data_path, target_size=(299, 299),
    batch_size=BATCH_SIZE, class_mode="categorical", shuffle=False)

In [ ]:
# Definir parámetros del entrenamiento
num_epochs = 200
opt = Adam(learning_rate=0.0001)

In [ ]:
# Resumen del modelo
model.summary()

In [ ]:
# Definir pasos por época
n_steps = train_data.samples // BATCH_SIZE
n_val_steps = test_data.samples // BATCH_SIZE
n_steps, n_val_steps

# Compilar el modelo
model.compile(
    optimizer=opt,
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

In [ ]:
from tf_explain.callbacks.grad_cam import GradCAMCallback

# Especificar el directorio de salida para guardar las visualizaciones
output_dir = "/content/drive/MyDrive/Colab_Notebooks/EcoScan/gradcam"  # Puedes cambiar esto al directorio que prefieras

# Extraer un batch de imágenes y etiquetas del generador
X_test_sample, y_test_sample = next(test_data)

# Asegurarte de que los datos de validación estén en el formato correcto
validation_data = (X_test_sample, y_test_sample)

# Crear un GradCAMCallback para cada clase (0 a 5, porque hay 6 clases en total)
callbacks = []

# Configurar un GradCAMCallback para cada clase
for class_index in range(6):
    callbacks.append(
        GradCAMCallback(
            validation_data=validation_data,
            layer_name='block14_sepconv2_act',  # Asegúrate de que sea el nombre correcto en tu modelo Xception
            class_index=class_index,
            output_dir=os.path.join(output_dir, f"class_{class_index}")
        )
    )

# Callbacks
early = EarlyStopping(monitor='val_loss', patience=15, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=1e-6)

# Añadir otros callbacks como EarlyStopping y ReduceLROnPlateau
callbacks.extend([early, reduce_lr])

In [ ]:
!pip install mlflow --quiet
!pip install pyngrok --quiet

In [ ]:
from mlflow.tracking import MlflowClient
import mlflow
from pyngrok import ngrok

# Configurar la URI de almacenamiento de MLflow
local_registry = "sqlite:///mlruns.db"
print(f"Running local model registry={local_registry}")
mlflow.set_tracking_uri(local_registry)

# Nombre del modelo que se registrará
model_name = "ClasificadorResiduos"

# Configura el nombre del experimento
experiment_name = "Clasificacion_Residuos_Proyecto"
mlflow.set_experiment(experiment_name)

In [ ]:
from mlflow import log_metric, log_param, log_artifacts
import shutil

with mlflow.start_run(run_name="Xception_Model_Training") as run:
    # Registrar los hiperparámetros
    mlflow.log_param("learning_rate", 0.0001)
    mlflow.log_param("batch_size", BATCH_SIZE)
    mlflow.log_param("num_epochs", num_epochs)

    # Entrenar el modelo
    result = model.fit(
        train_data,
        epochs=num_epochs,
        steps_per_epoch=n_steps,
        validation_data=test_data,
        validation_steps=n_val_steps,
        callbacks=callbacks
    )

    # Registrar las métricas finales
    final_accuracy = result.history['val_accuracy'][-1]
    final_loss = result.history['val_loss'][-1]
    mlflow.log_metric("val_accuracy", final_accuracy)
    mlflow.log_metric("val_loss", final_loss)

    # Registrar el modelo en MLflow
    mlflow.keras.log_model(
        model,
        artifact_path="xception-model",
        registered_model_name=model_name
    )

    # Crear un archivo de salida de ejemplo
    if not os.path.exists("outputs"):
        os.makedirs("outputs")
    with open("outputs/test.txt", "w") as f:
        f.write("El modelo fue registrado exitosamente en el almacén local de MLflow.")
    log_artifacts("outputs")
    shutil.rmtree('outputs')


In [ ]:
# Ejecutar la interfaz de usuario de MLflow en segundo plano
get_ipython().system_raw("mlflow ui --backend-store-uri sqlite:///content/mlruns.db --port 5000 &")

# Configura pyngrok para crear un túnel a la interfaz de MLflow
ngrok.kill()  # Termina cualquier túnel abierto
NGROK_AUTH_TOKEN = "2oHZxkpk4odzZvvnGecEiF20nhb_5y5mVAyrwnP4FkKTP3juE"  # Reemplaza con tu token de ngrok
if NGROK_AUTH_TOKEN:
    ngrok.set_auth_token(NGROK_AUTH_TOKEN)
ngrok_tunnel = ngrok.connect(addr="5000", proto="http", bind_tls=True)
print("MLflow Tracking UI:", ngrok_tunnel.public_url)

In [ ]:
# Gráfica de precisión (Accuracy)
plt.figure(figsize=(10, 5))
plt.plot(result.history["accuracy"], label="Entrenamiento - Precisión")
plt.plot(result.history["val_accuracy"], label="Validación - Precisión")
plt.title("Precisión del Modelo durante el Entrenamiento y Validación (Xception Final)")
plt.xlabel("Épocas")
plt.ylabel("Precisión")
plt.legend()
plt.grid(True)
plt.show()

# Gráfica de pérdida (Loss)
plt.figure(figsize=(10, 5))
plt.plot(result.history["loss"], label="Entrenamiento - Pérdida")
plt.plot(result.history["val_loss"], label="Validación - Pérdida")
plt.title("Pérdida del Modelo durante el Entrenamiento y Validación (Xception Final)")
plt.xlabel("Épocas")
plt.ylabel("Pérdida")
plt.legend()
plt.grid(True)
plt.show()

# Gráfico combinado de precisión y pérdida
plt.figure(figsize=(12, 6))
plt.plot(result.history['accuracy'], label="Precisión - Entrenamiento")
plt.plot(result.history['val_accuracy'], label="Precisión - Validación")
plt.plot(result.history['loss'], label="Pérdida - Entrenamiento")
plt.plot(result.history['val_loss'], label="Pérdida - Validación")
plt.title("Precisión y Pérdida del Modelo (Xception Final)")
plt.xlabel("Épocas")
plt.ylabel("Precisión / Pérdida")
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
# Hacer predicciones con el modelo
predictions = model.predict(test_data)

# Obtener etiquetas predichas
predicted_labels = np.argmax(predictions, axis=1)

# Etiquetas verdaderas
true_labels = test_data.classes

# Matriz de confusión
conf_matrix = confusion_matrix(true_labels, predicted_labels)

# Graficar matriz de confusión con seaborn
plt.figure(figsize=(8, 6))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues',
            xticklabels=test_data.class_indices.keys(),
            yticklabels=test_data.class_indices.keys())
plt.xlabel('Predicted')
plt.ylabel('True')
plt.title('Confusion Matrix (Xception Final)')
plt.show()

# Reporte de clasificación
class_names = list(test_data.class_indices.keys())
print(classification_report(true_labels, predicted_labels, target_names=class_names))

### Saliency Maps para el mapa de calor


In [ ]:
from tensorflow.keras import backend as K

# Función para calcular el mapa de saliencia para una clase específica usando SmoothGrad
def smooth_grad(model, img_tensor, class_index, noise_level=0.1, num_samples=20):
    """Genera un mapa de saliencia usando SmoothGrad."""
    smooth_saliency = np.zeros(img_tensor.shape[:2])
    for _ in range(num_samples):
        noise = np.random.normal(0, noise_level, img_tensor.shape)
        noisy_img = img_tensor + noise
        saliency = compute_saliency_map(model, noisy_img, class_index)
        smooth_saliency += saliency
    smooth_saliency /= num_samples  # Promedio
    return smooth_saliency

# Función básica de cálculo de saliencia (utilizada dentro de SmoothGrad)
def compute_saliency_map(model, img_tensor, label_index):
    """Calcula el mapa de saliencia para una imagen y una clase específica."""
    img_tensor = tf.convert_to_tensor([img_tensor])  # Agregar dimensión batch
    with tf.GradientTape() as tape:
        tape.watch(img_tensor)
        preds = model(img_tensor)
        loss = preds[:, label_index]
    grads = tape.gradient(loss, img_tensor)
    saliency = K.abs(grads)[0]
    saliency = np.max(saliency, axis=-1)
    return saliency

# Mostrar mapas de saliencia usando SmoothGrad para cada clase
class_labels = train_data.class_indices.keys()  # Asegúrate de tener acceso a los nombres de las clases

for class_index, class_name in enumerate(class_labels):
    # Obtener una imagen de muestra de la clase actual
    for images, labels in train_data:
        img_tensor = images[0]  # Primera imagen del lote
        label = labels[0].argmax()  # Obtener el índice de la clase (sin usar .numpy())

        # Asegurarse de que la imagen sea de la clase deseada
        if label == class_index:
            # Generar el mapa de saliencia usando SmoothGrad
            smooth_saliency = smooth_grad(model, img_tensor, class_index)

            # Normalizar la imagen para la visualización (escala [0, 1])
            img_display = (img_tensor - img_tensor.min()) / (img_tensor.max() - img_tensor.min())

            # Visualizar la imagen y el mapa de saliencia
            plt.figure(figsize=(12, 5))
            plt.subplot(1, 2, 1)
            plt.imshow(img_display)
            plt.title(f'Imagen original - Clase: {class_name}')
            plt.axis('off')

            plt.subplot(1, 2, 2)
            plt.imshow(smooth_saliency, cmap='hot')
            plt.title(f'SmoothGrad (Xception Final) - Clase: {class_name}')
            plt.axis('off')
            plt.show()
            break  # Solo una imagen por clase

SHAP para Explainable AI

In [ ]:
!pip install lime

In [ ]:
import lime
from lime import lime_image
import matplotlib.pyplot as plt
from skimage.segmentation import mark_boundaries
import numpy as np

# Crear un explicador LIME para imágenes
explainer = lime_image.LimeImageExplainer()

# Diccionario para traducir las etiquetas al español
class_translation = {
    'battery': 'e-waste',
    'glass': 'vidrio',
    'metal': 'metal',
    'organic': 'orgánico',
    'paper': 'papel',
    'plastic': 'plástico'
}

# Configuración: número de ejemplos por clase
num_classes = 6
examples_per_class = 5  # Puedes ajustar esto para ver más ejemplos por clase
class_examples = {i: 0 for i in range(num_classes)}  # Contador de ejemplos por clase

# Obtener un batch del conjunto de datos de validación
X_test_sample, y_test_sample = next(test_data)

# Recorrer las imágenes en el batch
for i in range(len(X_test_sample)):
    image = X_test_sample[i]
    label = np.argmax(y_test_sample[i])  # Convertir la etiqueta one-hot a un valor entero

    # Traducir la etiqueta al español
    class_name = list(class_translation.values())[label]  # Obtener el nombre traducido

    # Si aún no hemos generado suficientes ejemplos para esta clase
    if class_examples[label] < examples_per_class:
        # Explicar la predicción del modelo para esta imagen
        explanation = explainer.explain_instance(
            image,
            model.predict,
            top_labels=5,
            hide_color=0,
            num_samples=1000
        )

        # Obtener la imagen explicada y la máscara
        temp, mask = explanation.get_image_and_mask(
            label,
            positive_only=True,
            num_features=5,
            hide_rest=False
        )

        # Mostrar la explicación visual con LIME
        plt.figure(figsize=(8, 8))
        plt.imshow(mark_boundaries(temp, mask))
        plt.title(f"Explicación con LIME - {class_name}")
        plt.axis('off')
        plt.show()

        # Incrementar el contador para esta clase
        class_examples[label] += 1

    # Terminar si hemos alcanzado el número deseado de ejemplos por clase
    if all(count >= examples_per_class for count in class_examples.values()):
        break

In [ ]:
# Guardar el modelo final
model.save('/content/drive/MyDrive/Colab_Notebooks/EcoScan/wastenet_xception_final.h5')

In [ ]:
import os
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing import image

# Path a las imágenes reales para prueba
test_path = '/content/drive/MyDrive/Colab_Notebooks/EcoScan/test_real'

# Obtener los nombres de las imágenes
image_files = [os.path.join(test_path, img) for img in os.listdir(test_path)]

# Mostrar todas las imágenes
def display_images(image_paths):
    plt.figure(figsize=(15, 15))  # Ajusta el tamaño de la visualización
    for i, img_path in enumerate(image_paths):
        img = image.load_img(img_path)
        plt.subplot(5, 5, i + 1)  # Crea una grilla 5x5 (ajústalo según la cantidad de imágenes)
        plt.imshow(img)
        plt.axis('off')
    plt.tight_layout()
    plt.show()

# Mostrar las imágenes del directorio
display_images(image_files)

In [ ]:
import time
import numpy as np
from tensorflow.keras.applications.xception import preprocess_input
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image

# Cargar el modelo entrenado
model = load_model('/content/drive/MyDrive/Colab_Notebooks/EcoScan/wastenet_xception.h5')

# Diccionario para traducir las etiquetas al español
class_translation = {
    'battery': 'e-waste',
    'glass': 'vidrio',
    'metal': 'metal',
    'organic': 'orgánico',
    'paper': 'papel',
    'plastic': 'plástico'
}

# Lista para almacenar los tiempos de clasificación
classification_times = []

# Función para predecir la clase de una imagen y mostrarla junto con su etiqueta
def classify_and_display_image(image_path):
    # Cargar y mostrar la imagen
    img = image.load_img(image_path, target_size=(299, 299))
    plt.imshow(img)
    plt.axis('off')

    # Convertir la imagen a array y preprocesarla
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array = preprocess_input(img_array)

    # Tomar el tiempo de inicio
    start_time = time.time()

    # Hacer la predicción
    predictions = model.predict(img_array)

    # Tomar el tiempo de fin y calcular el tiempo
    end_time = time.time()
    classification_time = end_time - start_time
    classification_times.append(classification_time)

    # Obtener la clase predicha (índice) y traducirla al español
    predicted_class_idx = np.argmax(predictions[0])
    predicted_class = list(class_translation.keys())[predicted_class_idx]
    translated_class = class_translation[predicted_class]

    # Mostrar la etiqueta traducida
    plt.title(f'Clasificación (Xception Final): {translated_class}')
    plt.show()

# Clasificar y mostrar cada imagen
for image_file in image_files:
    classify_and_display_image(image_file)

# Calcular el tiempo promedio de clasificación
average_time = np.mean(classification_times)
print(f'Tiempo promedio de clasificación por imagen (Xception Final): {average_time:.4f} segundos')

In [ ]:
from tensorflow.keras import backend as K

# Función para calcular SmoothGrad
def smooth_grad(model, img_tensor, class_index, noise_level=0.1, num_samples=20):
    smooth_saliency = np.zeros(img_tensor.shape[:2])
    for _ in range(num_samples):
        noise = np.random.normal(0, noise_level, img_tensor.shape)
        noisy_img = img_tensor + noise
        saliency = compute_saliency_map(model, noisy_img, class_index)
        smooth_saliency += saliency
    smooth_saliency /= num_samples
    return smooth_saliency

# Función básica de cálculo de saliencia
def compute_saliency_map(model, img_tensor, label_index):
    img_tensor = tf.convert_to_tensor([img_tensor])
    with tf.GradientTape() as tape:
        tape.watch(img_tensor)
        preds = model(img_tensor)
        loss = preds[:, label_index]
    grads = tape.gradient(loss, img_tensor)
    saliency = K.abs(grads)[0]
    saliency = np.max(saliency, axis=-1)
    return saliency

# Función para predecir, mostrar la clasificación y el mapa de saliencia de cada imagen
def classify_and_display_with_saliency(image_path):
    # Cargar y mostrar la imagen original
    img = image.load_img(image_path, target_size=(299, 299))
    img_array = image.img_to_array(img)
    img_array = preprocess_input(np.expand_dims(img_array, axis=0))

    # Realizar predicción
    predictions = model.predict(img_array)
    predicted_class_idx = np.argmax(predictions[0])
    predicted_class = list(class_translation.keys())[predicted_class_idx]
    translated_class = class_translation[predicted_class]

    # Calcular el mapa de saliencia usando SmoothGrad
    smooth_saliency = smooth_grad(model, img_array[0], predicted_class_idx)

    # Visualizar la imagen, el nombre de la clase y el mapa de saliencia
    plt.figure(figsize=(12, 6))

    # Imagen original
    plt.subplot(1, 2, 1)
    plt.imshow(image.array_to_img(img_array[0] * 0.5 + 0.5))  # Escala de [-1,1] a [0,1]
    plt.title(f'Clasificación (Xception Final): {translated_class}')
    plt.axis('off')

    # Mapa de Saliencia
    plt.subplot(1, 2, 2)
    plt.imshow(smooth_saliency, cmap='hot')
    plt.title(f'SmoothGrad (Xception Final) - Clase: {translated_class}')
    plt.axis('off')

    plt.show()

# Obtener nombres de las imágenes y aplicar la clasificación y saliencia
image_files = [os.path.join(test_path, img) for img in os.listdir(test_path) if img.endswith(('.png', '.jpg', '.jpeg'))]
for image_file in image_files:
    classify_and_display_with_saliency(image_file)

In [ ]:
# Importar e iniciar TensorBoard en Google Colab
%reload_ext tensorboard
%tensorboard --logdir "/content/drive/MyDrive/Colab_Notebooks/EcoScan/gradcam"